In [1]:
import datetime
import pickle
from StringIO import StringIO

import numpy as np
import pandas as pd
import quandl
from talib import abstract
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import time
import urllib
import urllib2

from BeautifulSoup import BeautifulSoup

def str2datetime(date_str):
    return datetime.datetime.strptime(date_str, '%Y-%m-%d')

def get(url, data_list=None, timeout=10, max_try=3):

    if data_list:
        url = "{}?{}".format(url, urllib.urlencode(data_list))
    query = urllib2.Request(url)
    current_try = 0
    while current_try < max_try:
        try:
            response = urllib2.urlopen(query, timeout=timeout)
            html = response.read()
            response.close()
            return html
        except Exception, e:
            print e
            current_try += 1
            time.sleep(timeout)
    raise Exception("Cannot open page {}".format(url))


quandl.ApiConfig.api_key = "RYdPmBZoFyLXxg1RQ3fY"


def get_hk_interest_rate(detail_date):
    url = "http://www.hkab.org.hk/hibor/listRates.do"
    if not isinstance(detail_date, datetime.datetime):
        detail_date = str2datetime(detail_date)
    data_list = [('lang', 'en'), ('Submit', 'Search'), ('year', detail_date.year), ('month', detail_date.month),
                 ('day', detail_date.day)]
    page_html = get(url, data_list)
    soup = BeautifulSoup(page_html)
    rate_info = {"Overnight": np.nan,
                 "1 Week": np.nan,
                 # "2 Weeks": np.nan,
                 "1 Month": np.nan,
                 "2 Months": np.nan,
                 "3 Months": np.nan,
                 # "4 Months": np.nan,
                 # "5 Months": np.nan,
                 "6 Months": np.nan,
                 # "7 Months": np.nan,
                 # "8 Months": np.nan,
                 # "9 Months": np.nan,
                 # "10 Months": np.nan,
                 # "11 Months": np.nan,
                 "12 Months": np.nan,
                 }
    for table in soup('table'):
        if table.get('class') == 'etxtmed' and table.get('bgcolor') == '#ffffff':
            break

    else:
        return None

    td_list = table('td')
    for i in range(len(td_list)):
        if td_list[i].text in rate_info:
            rate_info[td_list[i].text] = float(td_list[i + 1].text)

    return rate_info


def get_yahoo_finance_data(symbol, start_date=None, end_date=None, remove_zero_volume=True):
    """
    Using yahoo finance API Get stock price with high low open close data

    :param symbol: stock symbol used in yahoo finance
    :param start_date: start date of the given stock data 2012-03-15
    :param end_date: end data
    :param remove_zero_volume: if True, will remove all data with zero volume
    :return: a list of stock price as [date, open, high, low, close]
    """
    data_list = [('s', symbol)]
    if hasattr(start_date, 'split'):
        start_date = str2datetime(start_date)

    if hasattr(end_date, 'split'):
        end_date = str2datetime(end_date)

    if start_date:
        data_list.append(('a', start_date.month - 1))
        data_list.append(('b', start_date.day))
        data_list.append(('c', start_date.year))
    if end_date:
        data_list.append(('d', end_date.month - 1))
        data_list.append(('e', end_date.day))
        data_list.append(('f', end_date.year))
    data_list.append(('g', 'd'))
    data_list.append(('ignore', '.csv'))

    url = "http://chart.finance.yahoo.com/table.csv"
    stock_info = get(url=url, data_list=data_list)
    stock_data = StringIO(stock_info)
    stock_df = pd.read_csv(stock_data)
    stock_df['Date'] = stock_df['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    stock_df = stock_df.set_index('Date').sort_index()

    if remove_zero_volume:
        return stock_df[stock_df['Volume'] > 0]
    else:
        return stock_df


def query_quandl_data(query_info, start_date=None, end_date=None, transform=None):
    """
    query info from Quandl

    :param query_info: target info to query
    :param start_date: start date
    :param end_date: end date
    :param transform: The following are useful transform types
        none	no effect	y"[t] = y[t]
        diff	row-on-row change	y"[t] = y[t] – y[t-1]
        rdiff	row-on-row % change	y"[t] = (y[t] – y[t-1]) / y[t-1]
        rdiff_from	latest value as % increment	y"[t] = (y[latest] – y[t]) / y[t]
        cumul	cumulative sum	y"[t] = y[0] + y[1] + … + y[t]
        normalize	scale series to start at 100	y"[t] = y[t] ÷ y[0] * 100
    :return: query result
    """
    data = quandl.get(query_info, start_date=start_date, end_date=end_date, returns='pandas', transform=transform)
    return data

In [5]:
from pandas.tseries.offsets import CustomBusinessDay
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday

with open('/Users/warn/PycharmProjects/Dissertation/data/hk_cal_rules.p') as f:
    rules = pickle.load(f)
    
b_day = CustomBusinessDay(calendar=AbstractHolidayCalendar('HK', rules=rules))

In [20]:
# data_df = data_df.reset_index().drop_duplicates(subset='index', keep='last').set_index('index')

hibor_df = pd.read_pickle('/Users/warn/PycharmProjects/Dissertation/data/HIBOR.p')
# hibor_df.index

In [22]:
for key in hibor_df.keys():
    hibor_df[key] = hibor_df[key].apply(float)
    
hibor_df.to_pickle('/Users/warn/PycharmProjects/Dissertation/data/HIBOR.p')


In [23]:
df = pd.read_pickle('/Users/warn/PycharmProjects/Dissertation/src/test.p')

In [24]:
df

,Open,High,Low,Close,Volume,MACD_12_26_9,MACD_7_14_9,SMA_3,SMA_13,SMA_21,...,RSI_14,RSI_21,Hang Seng Index,000001.SS,USD2HKD,EUR2HKD,12 Months,6 Months,Overnight,gold price in CNY
Date,,,,,,,,,,,,,,,,,,,,,
2012-08-30,75.2598,75.6172,73.1871,74.1877,11366400,0.794386,-0.341973,75.736267,77.326992,77.230395,...,43.644240,49.220418,19552.910156,2052.58,7.7567,12.2649,0.87000,0.56000,0.09857,10545.80000
2012-08-31,74.1163,75.6172,74.1163,75.4028,7549900,0.646550,-0.385969,75.283633,76.991631,77.339305,...,49.077530,52.488637,19482.570312,2047.52,7.7563,12.3186,0.87000,0.56000,0.09929,10468.00000
2012-09-03,75.2598,76.7607,75.0454,76.4034,4527900,0.603177,-0.284964,75.331300,76.837692,77.441410,...,53.088394,54.993232,19559.210938,2059.15,7.7571,12.3315,0.86357,0.56000,0.09929,10725.80000
2012-09-04,76.4034,76.9751,75.6887,75.8316,3675900,0.516707,-0.276522,75.879267,76.656262,77.390357,...,50.634028,53.307076,19429.910156,2043.65,7.7562,12.3207,0.86179,0.56000,0.09929,10772.60000
2012-09-05,76.1175,76.1175,74.5451,74.8310,4678700,0.363252,-0.378554,75.688667,76.359377,77.226995,...,46.576104,50.464077,19145.070312,2037.68,7.7572,12.3448,0.86179,0.56000,0.09929,10731.50000
2012-09-06,74.0448,74.9739,74.0448,74.1877,3832300,0.187567,-0.507308,74.950100,76.046000,77.036400,...,44.127554,48.710376,19209.300781,2051.92,7.7570,12.3592,0.86107,0.56000,0.09929,10791.10000
2012-09-07,75.4028,77.2610,75.1169,77.1181,7178900,0.281548,-0.232208,75.378933,75.985523,76.927490,...,55.582604,56.020521,19802.160156,2127.76,7.7556,12.4291,0.86821,0.56000,0.09929,10962.40000
2012-09-10,77.5469,78.4761,76.9037,78.1187,5306400,0.431791,0.071085,76.474833,76.073492,76.937700,...,58.696503,58.158609,19827.169922,2134.89,7.7554,12.4172,0.87000,0.56000,0.09929,10986.10000
2012-09-11,77.6899,78.6190,76.9751,78.2616,4658200,0.555982,0.282527,77.832800,76.144962,76.924086,...,59.137095,58.461431,19857.880859,2120.55,7.7542,12.4556,0.87000,0.56000,0.09929,11004.00000


In [32]:
transformer = MinMaxScaler(feature_range=(-1, 1))
transposed = transformer.fit_transform(df)
transposed_df = pd.DataFrame(transposed, index=df.index, columns=df.keys())
transposed_df.to_csv('transposed.csv')

In [31]:
mean = df.mean()
for key in df.keys():

    df[key] = df[key].replace(np.nan, mean[key])
    
df[df['000001.SS'].isnull()]

,Open,High,Low,Close,Volume,MACD_12_26_9,MACD_7_14_9,SMA_3,SMA_13,SMA_21,...,RSI_14,RSI_21,Hang Seng Index,000001.SS,USD2HKD,EUR2HKD,12 Months,6 Months,Overnight,gold price in CNY
Date,,,,,,,,,,,,,,,,,,,,,


In [34]:
df['Close'].shift(-1)

Date
2012-08-30     75.4028
2012-08-31     76.4034
2012-09-03     75.8316
2012-09-04     74.8310
2012-09-05     74.1877
2012-09-06     77.1181
2012-09-07     78.1187
2012-09-10     78.2616
2012-09-11     79.8340
2012-09-12     79.5481
2012-09-13     81.4779
2012-09-14     81.5493
2012-09-17     81.6208
2012-09-18     82.1211
2012-09-19     80.6917
2012-09-20     80.6917
2012-09-21     80.9776
2012-09-24     81.5493
2012-09-25     81.1920
2012-09-26     81.7637
2012-09-27     81.2635
2012-09-28     82.8358
2012-10-03     82.8358
2012-10-04     82.6929
2012-10-05     82.4070
2012-10-08     82.4785
2012-10-09     82.5499
2012-10-10     81.2635
2012-10-11     80.9061
2012-10-12     81.1920
                ...   
2016-08-19     98.0500
2016-08-22     99.0000
2016-08-23     98.8000
2016-08-24     98.0000
2016-08-25     98.7500
2016-08-26     98.8500
2016-08-29     99.1500
2016-08-30     99.7000
2016-08-31     99.5500
2016-09-01    102.8000
2016-09-02    103.2000
2016-09-05    102.2000
2016-0

In [19]:
mean.keys()

Index([u'Open', u'High', u'Low', u'Close', u'Volume', u'MACD_12_26_9',
       u'MACD_7_14_9', u'SMA_3', u'SMA_13', u'SMA_21', u'EMA_5', u'EMA_13',
       u'EMA_21', u'ROC_13', u'ROC_21', u'RSI_9', u'RSI_14', u'RSI_21',
       u'Hang Seng Index', u'000001.SS', u'USD2HKD', u'EUR2HKD',
       u'gold price in CNY'],
      dtype='object')